In [3]:


import os
import pandas as pd
import re
from lxml import etree
import datetime
from datetime import datetime
import time

file = "../data/output/extractAllPersons.log"

ns = {'tei': 'http://www.tei-c.org/ns/1.0', "xml": "http://www.w3.org/XML/1998/namespace"}
class bcolors:
    HEADER = '\033[95m'
    OKBLUE = '\033[94m'
    OKGREEN = '\033[92m'
    WARNING = '\033[93m'
    FAIL = '\033[91m'
    ENDC = '\033[0m'
    BOLD = '\033[1m'
    UNDERLINE = '\033[4m'

def readlist(pathtofile, filename):
      tree = etree.parse(pathtofile+"/"+filename)
      root = tree.getroot()
      return root

def readfiles(pathtofiles, personlistxml):
    rolenormlist = getrolenormlist()
    listofpersons = []
    listoferrors =[]
    for filename in os.listdir(pathtofiles):
       print("--------------------------")
       print(filename)
       regesttree = etree.parse(pathtofiles+"/"+filename)
       regestroot = regesttree.getroot()
       actualregest = filename
       creationdate = regestroot.find(".//tei:creation/tei:date", ns)
       origindate = regestroot.find(".//tei:history/tei:origin", ns)
       if creationdate is not None:
           date = createdate(creationdate)
       elif origindate is not None:
           date = createdate(origindate)
       else:
           date = " "
       personNodes = regestroot.findall(".//tei:body//*[@type='person']", ns)
       for rs in personNodes:
           try:
               person = createnewperson(rs, personlistxml, rolenormlist)
               person.append(actualregest)
               person.append(date)
               person.append(getroledead(rs))
               listofpersons.append(person)
           except:
               errorarray = [filename, rs.tag, rs.attrib, rs.xpath("normalize-space(.//text())")]
               listoferrors.append(errorarray)

    writeerrorfile(listoferrors)


    refs = []
    for person in listofpersons:
        if person[0] not in refs:
            refs.append(person[0])
    for ref in refs:
       death = computedeath(ref, listofpersons)
       for person in listofpersons:
           if ref in person:
               person.append(death)

    createcoloumns(listofpersons)


def writeerrorfile(listoferrors):
    with open("../Errors/AllPersons_ERROR.txt", "w") as file:
        if len(listoferrors) >0:
            file.write(datetime.fromtimestamp(time.time()).strftime('%Y-%m-%d %H:%M:%S'))
            file.write("\n")
            for error in listoferrors:
                file.write("Error in: %s \n" %error[0] )
                file.write("-Node %s \n" %error[1])
                file.write("-attributes: ")
                for attribute in error[2]:
                    file.write("%s; " %attribute)
                file.write("\n")
                file.write('-Nodetext: "%s"\n' %error[3])
                file.write("-------------\n")
        else:
            file.write(datetime.fromtimestamp(time.time()).strftime('%Y-%m-%d %H:%M:%S'))
            file.write("\nKeine ERRORS gefunden")




def computedeath(ref, personlist):
        dates =[]
        for person in personlist:
            if ref in person:
                if person[8] == True:
                    if len(person[7]) == 1:
                        date = person[7]
                        dates.append(person[7])
                    elif len(person[7]) == 2:
                        fromtodate = person[7]
                        dates.append(fromtodate[0])
        dates = sorted(dates)
        if len(dates)>0:
            if type(dates) == list:
                date = dates[0]
                return date
            else:
                return dates[0]
        else:
            return " "
def createcoloumns(listofpersons):
    personsFrame = pd.DataFrame()
    reflist = []
    regesttext = []
    sex = []
    regname = []
    title =[]
    title_norm =[]
    title_ref = []
    title_ref_norm = []
    title_off = []
    title_off_norm = []
    title_staff =[]
    title_staff_norm = []
    prof = []
    prof_norm =[]
    eventref = []
    regest = []
    date =[]
    fnrole = []
    death =[]
    dead = []
    fromdate=[]
    todate=[]
    year = []
    toyear =[]
    fromyear =[]


    for person in listofpersons:
        if person[0] != "": reflist.append(person[0])
        else: reflist.append(" ")

        if person[1] != "": regesttext.append(person[1])
        else: regesttext.append(" ")

        if len(person[2]) >0:
            if person[2][0] != "": sex.append(person[2][0])
            else: sex.append(" ")
            if person[2][1] != "": regname.append(person[2][1])
            else: regname.append(" ")
        else:
            sex.append(" ")
            regname.append(" ")

        titledict = person[3]
        if len(titledict['title']) >0:
            titlestring = "; ".join(titledict['title'])
            title.append(titlestring)
        else:
            title.append(" ")
        if len(titledict['prof']) >0:
            profstring = "; ".join(titledict['prof'])
            prof.append(profstring)
        else:
            prof.append(" ")
        if len(titledict['ref']) >0:
            title_refstring = "; ".join(titledict['ref'])
            title_ref.append(title_refstring)
        else:
            title_ref.append(" ")
        if len(titledict['off']) >0:
            title_offstring = "; ".join(titledict['off'])
            title_off.append(title_offstring)
        else:
            title_off.append(" ")
        if len(titledict['staff']) >0:
            title_staffstring = "; ".join(titledict['staff'])
            title_staff.append(title_staffstring)
        else:
            title_staff.append(" ")

        if len(titledict['title_norm']) >0:
            title_norm_string = "; ".join(titledict['title_norm'])
            title_norm.append(title_norm_string)
        else:
            title_norm.append(" ")
        if len(titledict['prof_norm']) >0:
            prof_norm_string = "; ".join(titledict['prof_norm'])
            prof_norm.append(prof_norm_string)
        else:
            prof_norm.append(" ")
        if len(titledict['ref_norm']) >0:
            ref_norm_string = "; ".join(titledict['ref_norm'])
            title_ref_norm.append(ref_norm_string)
        else:
            title_ref_norm.append(" ")
        if len(titledict['off_norm']) >0:
            off_norm_string = "; ".join(titledict['off_norm'])
            title_off_norm.append(off_norm_string)
        else:
            title_off_norm.append(" ")
        if len(titledict['staff_norm']) >0:
            print(titledict['staff_norm'])
            staff_norm_string = "; ".join(titledict['staff_norm'])
            title_staff_norm.append(staff_norm_string)
        else:
            title_staff_norm.append(" ")



        if len(person[4]) >0: eventref.append(person[4])
        else: eventref.append(" ")

        if len(person[5]) >0: fnrole.append(person[5])
        else: fnrole.append("no role given")

        if len(person[6]) >0: regest.append(person[6])
        else: regest.append(" ")


        if len(person[7]) == 2:
            persdate = person[7]
            date.append(persdate[0])
            fromdate.append("")
            todate.append("")
            year.append(persdate[0].year)
            toyear.append("")
            fromyear.append("")
        elif len(person[7]) == 3:
            fromtodate = person[7]
            date.append("")
            fromdate.append(fromtodate[0])
            todate.append(fromtodate[1])
            year.append("")
            toyear.append(fromtodate[0].year)
            fromyear.append(fromtodate[1].year)

        dead.append(person[8])

        death.append(person[9])

    personsFrame['ID'] = reflist
    personsFrame['text'] = regesttext
    personsFrame['reg_name'] = regname
    personsFrame['sex'] = sex
    personsFrame['titles'] = title
    personsFrame['titles_norm'] = title_norm
    personsFrame['titles_ref'] = title_ref
    personsFrame['titles_ref_norm'] = title_ref_norm
    personsFrame['prof'] = prof
    personsFrame['prof_norm'] = prof_norm
    personsFrame['staff'] = title_staff
    personsFrame['staff_norm'] = title_staff_norm
    personsFrame['off'] = title_off
    personsFrame['off_norm'] = title_off_norm
    personsFrame['fn_role'] = fnrole
    personsFrame['event_ref']  = eventref
    personsFrame['xml'] = regest
    personsFrame['date'] = date
    personsFrame['from'] = fromdate
    personsFrame['to'] = todate
    personsFrame['dead'] = dead
    personsFrame['earliest possible death'] = death
    personsFrame['year'] = year
    personsFrame['fromyear'] = fromyear
    personsFrame['toyear'] = toyear

    personsFrame.to_csv("../data/AllPersons.csv", sep="\t", encoding="utf-8")



def createnewperson(rs, personlistxml, rolenormlist):
        text = cleanuptextnodestostring(rs.xpath(".//text()"))
        print("--Text: %s" %text)
        roles = getpersonroles(rs, rolenormlist)
        rsrole = getrstyperole(rs)
        events = getpersonevent(rs)
        if rs.attrib["ref"] is not None:
            ref = rs.attrib["ref"].replace("#","")
        else:
            print(bcolors.FAIL+"<rs type='person'> ref nicht gefunden")

        person = getinformationsfrompersonlistxml(ref, personlistxml)
        return [ref, text, person, roles, events, rsrole]

def getrstyperole(persnode):
    parentrs = persnode.xpath("parent::*[@type='fn']")
    print("--@fn-Parents")
    for parent in parentrs:
        print("---Node: %s" %parent.tag)
        print("----Attribute: %s" %parent.attrib)
    if len(parentrs) >0:
        return parentrs[0].attrib['role']
    else:
        return "None"

def getroledead(persnode):
    deadnode  = persnode.find("./tei:roleName[@type='dead']",ns)
    if deadnode is not None:
        return True
    else:
        return False

def getpersonroles(persnode, rolenormlist):
    roles = {}
    rolenamenodes = persnode.findall("./tei:roleName", ns)
    print("--roleNames:")
    if rolenamenodes is not None:
        roles_prof = []
        roles_title = []
        roles_ref = []
        roles_off = []
        roles_staff= []
        roles_prof_norm = []
        roles_title_norm = []
        roles_ref_norm = []
        roles_off_norm = []
        roles_staff_norm= []
        for rolenamenode in rolenamenodes:
            print("---Node: %s"%rolenamenode.tag)
            print("----Attribute: %s"%rolenamenode.attrib)
            content = cleanuptextnodestostring(rolenamenode.xpath(".//text()"))
            print("----Content: %s" %content)
            if rolenamenode.attrib['type'] == "prof":
                rolelist = normrole(rolenormlist, content)
                roles_prof.append(rolelist[0])
                roles_prof_norm.append(rolelist[1])
            elif rolenamenode.attrib['type'] == "title":
                rolelist = normrole(rolenormlist, content)
                roles_title.append(rolelist[0])
                roles_title_norm.append(rolelist[1])
            elif rolenamenode.attrib['type'] == "rep":
                rolelist = normrole(rolenormlist, content)
                roles_ref.append(rolelist[0])
                roles_ref_norm.append(rolelist[1])
            elif rolenamenode.attrib['type'] == "off":
                rolelist = normrole(rolenormlist, content)
                roles_off.append(rolelist[0])
                roles_off_norm.append(rolelist[1])
            elif rolenamenode.attrib['type'] == "staff":
                rolelist = normrole(rolenormlist, content)
                roles_staff.append(rolelist[0])
                roles_staff_norm.append(rolelist[1])

    roles['title'] = roles_title
    roles['ref'] = roles_ref
    roles['prof'] = roles_prof
    roles['off'] = roles_off
    roles['staff'] = roles_staff
    roles['title_norm'] = roles_title_norm
    roles['ref_norm'] = roles_ref_norm
    roles['prof_norm'] = roles_prof_norm
    roles['off_norm'] = roles_off_norm
    roles['staff_norm'] = roles_staff_norm
    return roles

def getpersonevent(persnode):
    eventnode = getparentevent(persnode)
    if eventnode is not None:
        if 'ref' in eventnode.attrib:
            event_ref = eventnode.attrib['ref']
            return event_ref
        else: return ""
    else: return ""


def getparentevent(node):
        actualnode = node
        parents = actualnode.xpath("ancestor::*[@type='event']")
        if len(parents) >0:
            return parents[0]
        else: return None




def createdate(node):
    if 'when' in node.attrib:
        when = node.attrib['when'].replace('-','')
        date = datetime.strptime(when, '%Y%m%d').date()
        year = str(date).split("-")[0]
        return [date,year]

    elif 'notAfter' in node.attrib:
        notafter = node.attrib['notAfter']
        if "-" in notafter:
            date = datetime.strptime(notafter, '%Y-%m-%d').date()
        else:
            date = datetime.strptime(notafter, '%Y').date()
        year = str(date).split("-")[0]
        return [date, year]
    elif 'from' in node.attrib:
        fr =node.attrib['from'].replace('-','')
        to = node.attrib['to'].replace('-','')
        fromdate = datetime.strptime(fr, '%Y%m%d').date()
        todate = datetime.strptime(to, '%Y%m%d').date()
        year = str(fromdate).split("-")[0]
        if fromdate == todate:
            return[fromdate,year]
        else:
            return [fromdate, todate,year]



def getinformationsfrompersonlistxml(ref, personlist):
    ref = ref.replace("#", '')
    person = personlist.find(".//tei:person[@xml:id='"+ref+"']", ns)
    if person is not None:
        if "sex" in person.attrib:
            sex = person.attrib['sex']
        else: sex = "-"
        persname = person.find(".//tei:persName", ns)
        text = cleanuptextnodestostring(persname.xpath(".//text()"))
        return [sex, text]
    else:
        return []



def cleanuptextnodestostring(textnodes):
    cleanedlist =[]
    for textnode in textnodes:
        textstring = str(textnode)
        tokenized = textstring.split(" ")
        cleandtokens = []
        for token in tokenized:
            if len(token) >1:
                token = re.sub("\n","", token)
                cleandtokens.append(token)
        cleanedlist += cleandtokens
    return " ".join(cleanedlist)

def getrolenormlist():
    role_mapping = pd.read_csv("https://docs.google.com/spreadsheets/d/1_ygvqRnDTg6Rx2wp-RNPvZ3cKQFxfDPo3d7SBDiLO5c/export?gid=0&format=csv", dtype=str)\
    .iloc[:,1:3]
    rolesnormlist = role_mapping.set_index("role").to_dict()

    return rolesnormlist['role_norm']


def normrole(rolenormlist, role):
    if role.lower() in rolenormlist:
        return [role, rolenormlist[role.lower()]]
    elif role in rolenormlist:
        return [role, rolenormlist[role]]
    else:
        return [role, " "]

personlistxml = readlist("../../indices/lists", "personList.xml")
readfiles("../../sources/Satzbuch_CD/1448/done", personlistxml)
print("done")

--------------------------
00054-eintrag_vom_1448-06-10.xml
--Text: Andre Vischel
--roleNames:
--@fn-Parents
---Node: {http://www.tei-c.org/ns/1.0}rs
----Attribute: {'type': 'fn', 'role': 'issuer'}
--Text: Barbara sein hausfrau
--roleNames:
---Node: {http://www.tei-c.org/ns/1.0}roleName
----Attribute: {'type': 'kin', 'corresp': 'pe__andreas_vischel_SB_CD_00054'}
----Content: hausfrau
--@fn-Parents
---Node: {http://www.tei-c.org/ns/1.0}rs
----Attribute: {'type': 'fn', 'role': 'issuer'}
--Text: Stephans Imbrung des kramer
--roleNames:
---Node: {http://www.tei-c.org/ns/1.0}roleName
----Attribute: {'type': 'prof'}
----Content: kramer
--@fn-Parents
--Text: hern Symon Pirkwicz caplan der Heiligen dreir Kunig altar zu der Himlporten
--roleNames:
---Node: {http://www.tei-c.org/ns/1.0}roleName
----Attribute: {'type': 'title'}
----Content: hern
---Node: {http://www.tei-c.org/ns/1.0}roleName
----Attribute: {'type': 'off', 'corresp': 'org__wien-st_agnes_auf_der_himmelpforte-altar_dreikoenige'}
---